In [ ]:
!sudo apt install gdal-bin python-gdal python3-gdal 
!sudo apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes 
!pip install folium 
!pip install plotly_express

In [1]:
import pandas as pd 
import numpy as np
import json




import folium



from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv('yelp_business.csv')
df.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [ ]:

df.shape

In [3]:
df['Restaurants'] = df['categories'].str.contains('Restaurants')


In [4]:
df_restaurants = df.loc[df.Restaurants == True]
df_restaurants.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,Restaurants
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,True
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants,True
10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",NaN,"""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,3,0,Breakfast & Brunch;Gluten-Free;Coffee & Tea;Fo...,True
14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional),True
15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",Yonge and Eglinton,"""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,12,0,Italian;French;Restaurants,True


In [ ]:
df_restaurants.shape

In [20]:
top_restaurants = df_restaurants.sort_values(by=['review_count', 'stars'], ascending=False)[:1000]
top_restaurants.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,Restaurants
97944,4JNXUYY8wbaaDmk3BPzlWw,"""Mon Ami Gabi""",The Strip,"""3655 Las Vegas Blvd S""",Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,French;Steakhouses;Restaurants;Breakfast & Brunch,True
119907,RESDUcs7fIiihp38-d6_6g,"""Bacchanal Buffet""",The Strip,"""3570 S Las Vegas Blvd""",Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...,True
69993,K7lWdNUhCbcnEvI0NhGewg,"""Wicked Spoon""",The Strip,"""3708 Las Vegas Blvd S""",Las Vegas,NV,89109,36.109538,-115.176170,3.5,5950,1,Buffets;Breakfast & Brunch;Restaurants,True
81212,cYwJA2A6I12KNkm2rtXd5g,"""Gordon Ramsay BurGR""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.110724,-115.172169,4.0,5447,0,American (Traditional);Burgers;Restaurants,True
139699,DkYS3arLOhA8si5uUEmHOw,"""Earl of Sandwich""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.108228,-115.171869,4.5,4869,1,Caterers;Sandwiches;Restaurants;Food Delivery ...,True


In [21]:
coords = top_restaurants[['longitude','latitude']]
distortions = []
K = range(1,100)
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    distortions.append(kmeansModel.inertia_)


In [22]:
from sklearn.metrics import silhouette_score

sil = []
kmax = 50

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(coords)
  labels = kmeans.labels_
  sil.append(silhouette_score(coords, labels, metric = 'euclidean'))

In [23]:
sil

[0.92801060747713837,
 0.95493530066898724,
 0.92414685127092089,
 0.94102372441048299,
 0.9494674213690556,
 0.96115991805085688,
 0.97322848909051096,
 0.97529105249265735,
 0.80766795969492067,
 0.57877311138644971,
 0.57051966229846796,
 0.61337897423111332,
 0.62312809585299256,
 0.63833390671675416,
 0.63257506582498713,
 0.60316880311146481,
 0.59232237852900227,
 0.59852723502431393,
 0.60293753302696851,
 0.59513197234568793,
 0.59676837956925544,
 0.5628706803029262,
 0.59122176991053732,
 0.54996964678024274,
 0.56445064639733455,
 0.5540856765969655,
 0.56577046294135858,
 0.53214686667281641,
 0.55883523654980427,
 0.5758342361151868,
 0.50327965221133741,
 0.52132580342185941,
 0.56454872292797798,
 0.56450924414825021,
 0.57731218051305377,
 0.5261113523238018,
 0.52464306577168707,
 0.51866758432739657,
 0.56166497336019128,
 0.54965122205814043,
 0.56416329664763609,
 0.52527389705854266,
 0.52515269516799601,
 0.49418061959318516,
 0.51372672213785175,
 0.535444135443

In [24]:
kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(coords)
y = kmeans.labels_
print("k = 5", " silhouette_score ", silhouette_score(coords, y, metric='euclidean'))

k = 5  silhouette_score  0.94102372441


In [25]:

top_restaurants['cluster'] = kmeans.predict(top_restaurants[['longitude','latitude']])
top_restaurants.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,Restaurants,cluster
97944,4JNXUYY8wbaaDmk3BPzlWw,"""Mon Ami Gabi""",The Strip,"""3655 Las Vegas Blvd S""",Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,French;Steakhouses;Restaurants;Breakfast & Brunch,True,1
119907,RESDUcs7fIiihp38-d6_6g,"""Bacchanal Buffet""",The Strip,"""3570 S Las Vegas Blvd""",Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...,True,1
69993,K7lWdNUhCbcnEvI0NhGewg,"""Wicked Spoon""",The Strip,"""3708 Las Vegas Blvd S""",Las Vegas,NV,89109,36.109538,-115.176170,3.5,5950,1,Buffets;Breakfast & Brunch;Restaurants,True,1
81212,cYwJA2A6I12KNkm2rtXd5g,"""Gordon Ramsay BurGR""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.110724,-115.172169,4.0,5447,0,American (Traditional);Burgers;Restaurants,True,1
139699,DkYS3arLOhA8si5uUEmHOw,"""Earl of Sandwich""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.108228,-115.171869,4.5,4869,1,Caterers;Sandwiches;Restaurants;Food Delivery ...,True,1


In [26]:
top_restaurants_order = top_restaurants.sort_values(by=['review_count', 'stars'], ascending=False)
top_restaurants_order.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,Restaurants,cluster
97944,4JNXUYY8wbaaDmk3BPzlWw,"""Mon Ami Gabi""",The Strip,"""3655 Las Vegas Blvd S""",Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,French;Steakhouses;Restaurants;Breakfast & Brunch,True,1
119907,RESDUcs7fIiihp38-d6_6g,"""Bacchanal Buffet""",The Strip,"""3570 S Las Vegas Blvd""",Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...,True,1
69993,K7lWdNUhCbcnEvI0NhGewg,"""Wicked Spoon""",The Strip,"""3708 Las Vegas Blvd S""",Las Vegas,NV,89109,36.109538,-115.176170,3.5,5950,1,Buffets;Breakfast & Brunch;Restaurants,True,1
81212,cYwJA2A6I12KNkm2rtXd5g,"""Gordon Ramsay BurGR""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.110724,-115.172169,4.0,5447,0,American (Traditional);Burgers;Restaurants,True,1
139699,DkYS3arLOhA8si5uUEmHOw,"""Earl of Sandwich""",The Strip,"""3667 Las Vegas Blvd S""",Las Vegas,NV,89109,36.108228,-115.171869,4.5,4869,1,Caterers;Sandwiches;Restaurants;Food Delivery ...,True,1


In [27]:
def recommend_restaurants(df, longitude, latitude):
    # Predict the cluster for longitude and latitude provided
    cluster = kmeans.predict(np.array([longitude,latitude]).reshape(1,-1))[0]
    print(cluster)
   
    # Get the best restaurant in this cluster
    return  df[df['cluster']==cluster].iloc[0:30][['name','city','stars', 'latitude','longitude'	]]

In [29]:
north_carolina=recommend_restaurants(top_restaurants,35.152725,-80.827953)
north_carolina_list=[]
north_carolina_list=north_carolina.values.tolist()
north_carolina


0


,name,city,stars,latitude,longitude
144620,"""The Cowfish Sushi Burger Bar""",Charlotte,4.0,35.152725,-80.827953
115558,"""Amélie's French Bakery & Café""",Charlotte,4.0,35.241145,-80.812057
73231,"""Midwood Smokehouse""",Charlotte,4.5,35.221004,-80.814875
92689,"""Tupelo Honey""",Charlotte,4.0,35.211178,-80.858604
43961,"""Cabo Fish Taco""",Charlotte,4.0,35.247251,-80.805827
13417,"""Soul Gastrolounge""",Charlotte,4.0,35.220214,-80.813368
173858,"""Pinky's Westside Grill""",Charlotte,4.0,35.229054,-80.867370
10315,"""Mert's Heart & Soul""",Charlotte,3.5,35.227318,-80.840130
134627,"""Viva Chicken""",Charlotte,4.5,35.213421,-80.826022
122612,"""Futo Buta""",Charlotte,4.0,35.215379,-80.855443


In [50]:
illinois=recommend_restaurants(top_restaurants,40.1105875,-88.2072697)
illinois_list=[]
illinois_list=illinois.values.tolist()
illinois
# illinois = df_restaurants[df_restaurants.state == 'IL'].iloc[0:30][['name','city','stars', 'latitude','longitude'	]].sort_values(by=[ 'stars'], ascending=False)

# illinois_list=[]
# illinois_list=illinois.values.tolist()
# illinois

,name,city,stars,latitude,longitude
1991,"""JP Cool's Bar & Grill""",Mansfield,5.0,40.211684,-88.505562
9722,"""Pies by Inge""",Monticello,5.0,40.025702,-88.573161
7397,"""Arby's""",Champaign,4.5,40.131140,-88.257562
8897,"""C & C Kitchen""",Rantoul,4.5,40.310654,-88.158583
4853,"""Proud Marys'""",Tuscola,4.5,39.791242,-88.273007
190,"""Zorba's Restaurant""",Champaign,4.0,40.110085,-88.229304
1341,"""I Wok""",Savoy,4.0,40.059012,-88.252238
8427,"""El Toro""",Urbana,3.5,40.123643,-88.200627
6352,"""Silver Mine Subs""",Champaign,3.5,40.108174,-88.229693
5752,"""Big Grove Tavern""",Champaign,3.5,40.118381,-88.242996


In [33]:
arizona=recommend_restaurants(top_restaurants,-111.928001,32.424564)
arizona_list=[]
arizona_list=arizona.values.tolist()
arizona

2


,name,city,stars,latitude,longitude
148820,"""Pizzeria Bianco""",Phoenix,4.0,33.449160,-112.065635
152492,"""Four Peaks Brewing""",Tempe,4.5,33.419568,-111.916097
100969,"""Bobby Q""",Phoenix,4.5,33.561085,-112.115747
115632,"""Lux Central""",Phoenix,4.5,33.500597,-112.074237
130242,"""Rehab Burger Therapy""",Scottsdale,4.5,33.491503,-111.925627
171047,"""Cibo""",Phoenix,4.5,33.454960,-112.079908
831,"""La Santisima""",Phoenix,4.0,33.469201,-112.047393
41036,"""The Mission Old Town""",Scottsdale,4.0,33.492287,-111.924273
163111,"""Joe's Farm Grill""",Gilbert,4.0,33.321824,-111.726057
157852,"""Citizen Public House""",Scottsdale,4.5,33.498210,-111.927963


In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient("34.83.108.19",27017)
db = client['recommendation']
    
for i in range(len(lasvegas_list)):
    article = {"State": "LasVegas",
            "Name": lasvegas_list[i][0],
            "City": lasvegas_list[i][1],
            "Rating":lasvegas_list[i][2]   
             "loc": [lasvegas_list[i][4],lasvegas_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)

In [35]:
for i in range(len(arizona_list)):
    article = {"State": "Arizona",
            "Name": arizona_list[i][0],
            "City": arizona_list[i][1],
             "Rating":arizona_list[i][2],  
             "loc": [arizona_list[i][4],arizona_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)

In [51]:
for i in range(len(illinois_list)):
    article = {"State": "Illinois",
            "Name": illinois_list[i][0],
            "City": illinois_list[i][1],
             "Rating":illinois_list[i][2],  
             "loc": [illinois_list[i][4],illinois_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)

In [45]:

colorado = df_restaurants[df_restaurants.state == 'CO'].iloc[0:2][['name','city','stars', 'latitude','longitude'	]]

colorado_list=[]
colorado_list=colorado.values.tolist()
colorado


,name,city,stars,latitude,longitude
898,"""The Abbey Bar & Restaurant""",Main Street,3.0,51.812438,-8.392635
6307,"""Sage Restaurant""",Main Street,4.5,51.916151,-8.174059


In [49]:
lasvegas = df_restaurants[df_restaurants.state == 'NV'].iloc[0:30][['name','city','stars', 'latitude','longitude'	]].sort_values(by=[ 'stars'], ascending=False)

lasvegas_list=[]
lasvegas_list=lasvegas.values.tolist()
lasvegas

,name,city,stars,latitude,longitude
538,"""Baja Fresh""",Las Vegas,5.0,36.159483,-115.287451
778,"""The Hummus Factory""",Las Vegas,4.5,36.143204,-115.262866
721,"""The Blacc Boxx""",North Las Vegas,4.5,36.196983,-115.180376
691,"""Mexicali Raspados""",Las Vegas,4.5,36.100688,-115.101083
177,"""Geebee's Bar & Grill""",Las Vegas,4.5,36.034244,-115.171409
274,"""Trattoria Italia""",Las Vegas,4.5,36.010086,-115.118656
303,"""Cancun Bar & Grill""",Las Vegas,4.5,36.098632,-115.136079
366,"""Kinthai""",Las Vegas,4.5,36.144023,-115.195896
52,"""Flight Deck Bar & Grill""",Las Vegas,4.0,36.066914,-115.170848
428,"""Rise & Shine - A Steak & Egg Place""",Henderson,4.0,36.024323,-115.062857


In [43]:
for i in range(len(lasvegas_list)):
    article = {"State": "LasVegas",
            "Name": lasvegas_list[i][0],
            "City": lasvegas_list[i][1],
            "Rating":lasvegas_list[i][2],   
             "loc": [lasvegas_list[i][4],lasvegas_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)

In [40]:
for i in range(len(colorado_list)):
    article = {"State": "Colorado",
            "Name": colorado_list[i][0],
            "City": colorado_list[i][1],
            "Rating":colorado_list[i][2],   
             "loc": [colorado_list[i][4],colorado_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)

In [32]:
import pymongo
from pymongo import MongoClient
client = MongoClient("34.83.108.19",27017)
db = client['recommendation']
    
for i in range(len(north_carolina_list)):
    article = {"State": "North Carolina",
            "Name": north_carolina_list[i][0],
            "City": north_carolina_list[i][1],
             "Rating":north_carolina_list[i][2],  
             "loc": [north_carolina_list[i][4],north_carolina_list[i][3]],}
    articles = db.articles
    result = articles.insert_one(article)